In [2]:
import keras
import numpy as np

In [2]:
# *********** Capa de vectores 1s o 0s ************* #
#                                                    #
# Toma un vector de tamaño S predicción, lo pasa a   #
# binario, y crea S vectores con valores 0s o 1s en  #
# cada uno.                                          #
# input : vector de tamaño S con valores flotantes.  #
# output : S vectores de tamaño S de valores         #
#          binarios, según el vector.                #

class LayerExpand(keras.layers.Layer):
    def __init__(self, stacks):
        super(LayerExpand,self).__init__(trainable=False)
        self.stacks = stacks

    def call(self, arr):
        list_arr = []
        categorical = self.analyser(arr)
        for stack in range(self.stacks):
            if categorical[stack] == 1:
                list_arr.append(np.full((1,self.stacks),1))
            else: 
                list_arr.append(np.full((1,self.stacks),0))

        return list_arr
    
    def analyser(self, arr):
        err = 0.75
        label = np.zeros((arr.shape[0]))
        
        for i in range(arr.shape[0]):
            if arr[i] >= err: label[i] = 1

        return label
    

In [6]:
vector = np.array([1.,0.,1.,0.,0.])

In [7]:
LayerExpand(5)(vector)

[array([[1, 1, 1, 1, 1]]),
 array([[0, 0, 0, 0, 0]]),
 array([[1, 1, 1, 1, 1]]),
 array([[0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 0]])]

In [5]:
# ************* Capa de desarmado ****************** #
#                                                    #
# Toma una entrada de predicciones flotantes, las    #
# transforma a binarias y luego crea r vectores      #
# según los r números 1 que aparezcan en el vector   #
# original.                                          #
# input : Vector de predicciones flotantes           #
# output : r vectores binarios.                      #

class LayerExpandLabel(keras.layers.Layer):
    def __init__(self, stacks):
        super(LayerExpandLabel,self).__init__(trainable=False)
        self.stacks = stacks
    
    def call(self, labels):
        return self.getLabels(labels)
    
    def getLabels(self, arr):
        list_ = []
        vector = self.analyser(arr)
        idx_labels = np.where(vector == 1)[0]
        for i in idx_labels:
            list_.append(self.to_labels(i, arr.shape[0]))

        return np.array(list_)

    def analyser(self, arr):
        err = 0.75
        label = np.zeros((arr.shape[0]))
        
        for i in range(arr.shape[0]):
            if arr[i] >= err: label[i] = 1

        return label
    
    def to_labels(self, idx, S):
        arr = np.zeros(S)
        arr[idx] = 1
        return arr 





In [97]:
# ********* Capa predictora de destino ************ #
# ************* múltiples salidas****************** #
#                                                   #
# Utiliza el modelo de destino para hacer las pre-  #
# dicciones acorde a los estados.                   #
#                                                   #
# input : N estados de S x (H + 2).                 #
# output : N predicciones de 1 x S.                 #


class ConcatenateLayer(keras.layers.Layer):
    def __init__(self):
        super(ConcatenateLayer,self).__init__()

    def call(self, state, labels):
        new_data = []

        for label in labels:
            copy_state = np.copy(state)
            matrix = []
            for i in range(copy_state.shape[0]):
                matrix.append(np.concatenate((copy_state[i],[label[i]])))
            new_data.append(np.array(matrix))

        return np.array(new_data)

In [135]:
# ******** Capa predictoria de destino ************ #
# ************* Multiples salidas****************** #
#                                                   #
# Utiliza el modelo de destino para hacer las pre-  #
# dicciones acorde a los estados.                   #
#                                                   #
# input : N estados de S x (H + 2).                 #
# output : N predicciones de 1 x S.                 #


class PredictLayer(keras.layers.Layer):
    def __init__(self, model):
        super(PredictLayer,self).__init__()
        self.model = model

    def call(self, states):
        
        return np.array(self.model(states))

In [14]:
class LayerExpandOutput(keras.layers.Layer):
    def __init__(self):
        super(LayerExpandOutput, self).__init__(trainable=False)

    def call(self, arr):
        list_arr = []
        for val in arr:
            list_arr.append(np.full((1,arr.shape[0]),val))
        return np.hstack(list_arr)[0]


# New Layers

In [15]:
LayerExpandOutput()(np.array([1,0,1]))

array([1, 1, 1, 0, 0, 0, 1, 1, 1])

In [16]:
class ConcatenateLayerState(keras.layers.Layer):
    def __init__(self):
        super(ConcatenateLayerState,self).__init__()

    def call(self, state, label):

        matrix = []
        for i in range(state.shape[0]):
            matrix.append(np.concatenate((state[i],[label[i]])))


        return np.array(matrix)

In [20]:
ConcatenateLayerState()(np.random.rand(5, 5), np.array([1,0,0,1,0]))

array([[0.88468987, 0.00551156, 0.99806726, 0.69934541, 0.3596051 ,
        1.        ],
       [0.37355953, 0.74380708, 0.97602624, 0.56751066, 0.14217599,
        0.        ],
       [0.96196651, 0.74498957, 0.61904216, 0.19567636, 0.46042445,
        0.        ],
       [0.07917506, 0.63422531, 0.53008091, 0.30416709, 0.99408752,
        1.        ],
       [0.65223879, 0.31226313, 0.37174976, 0.77130944, 0.43044087,
        0.        ]])